# Part I. ETL Pipeline for Pre-Processing the Files

#### Import Python packages 

In [1]:
# Import Python packages 
import pandas as pd
import cassandra
import re
import os
import glob
import numpy as np
import json
import csv

#### Creating list of filepaths to process original event csv data files

In [2]:
# checking your current working directory
print(os.getcwd())

# Get your current folder and subfolder event data
filepath = os.getcwd() + '/event_data'

# Create a for loop to create a list of files and collect each filepath
for root, dirs, files in os.walk(filepath):
    
# join the file path and roots with the subdirectories using glob
    file_path_list = glob.glob(os.path.join(root,'*'))
#     print(file_path_list)

/home/workspace


#### Processing the files to create the data file csv that will be used for Apache Casssandra tables

In [3]:
# initiating an empty list of rows that will be generated from each file
full_data_rows_list = [] 
    
# for every filepath in the file path list 
for f in file_path_list:
# reading csv file 
    with open(f, 'r', encoding = 'utf8', newline='') as csvfile: 
        # creating a csv reader object 
        csvreader = csv.reader(csvfile) 
        next(csvreader)
        
 # extracting each data row one by one and append it        
        for line in csvreader:
            #print(line)
            full_data_rows_list.append(line) 
            
# uncomment the code below if you would like to get total number of rows 
#print(len(full_data_rows_list))
# uncomment the code below if you would like to check to see what the list of event data rows will look like
#print(full_data_rows_list)

# creating a smaller event data csv file called event_datafile_full csv that will be used to insert data into the \
# Apache Cassandra tables
csv.register_dialect('myDialect', quoting=csv.QUOTE_ALL, skipinitialspace=True)

with open('event_datafile_new.csv', 'w', encoding = 'utf8', newline='') as f:
    writer = csv.writer(f, dialect='myDialect')
    writer.writerow(['artist','firstName','gender','itemInSession','lastName','length',\
                'level','location','sessionId','song','userId'])
    for row in full_data_rows_list:
        if (row[0] == ''):
            continue
        writer.writerow((row[0], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[12], row[13], row[16]))


In [4]:
# check the number of rows in your csv file
with open('event_datafile_new.csv', 'r', encoding = 'utf8') as f:
    print(sum(1 for line in f))

6821


# Part II. Complete the Apache Cassandra coding portion of your project. 

## The event_datafile_new.csv contains the following columns: 
- artist 
- firstName of user
- gender of user
- item number in session
- last name of user
- length of the song
- level (paid or free song)
- location of the user
- sessionId
- song title
- userId

#### Creating a Cluster

In [5]:
# Connecting to a Cassandra instance

from cassandra.cluster import Cluster
cluster = Cluster()

# Establishing connection using a session
session = cluster.connect()

#### Create Keyspace

In [6]:
# Creating a Keyspace 
session.execute("""CREATE KEYSPACE IF NOT EXISTS stream WITH REPLICATION = {'class' : 'SimpleStrategy', 'replication_factor' : 1 }""")

#### Set Keyspace

In [7]:
#Setting Keyspace for the session
session.set_keyspace('stream')

### Query 1: Give me the artist, song title and song's length in the music app history that was heard during  sessionId = 338, and itemInSession  = 4

#### Create table

Since we will be filtering with sessionId and itemInSession columns, we need to define them as partition keys.

In [8]:
drop_songs = """DROP TABLE IF EXISTS songs"""
session.execute(drop_songs)

create_songs = """CREATE TABLE IF NOT EXISTS songs (artist text, song text, length float, sessionId int, itemInSession int, primary key(sessionId,itemInSession))"""
session.execute(create_songs)


#### Insert Records

In [9]:
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        query = "INSERT INTO songs (artist, song, length, sessionId, itemInSession) "
        query = query + "VALUES (%s, %s, %s,%s,%s)"
        session.execute(query, (line[0], line[9], float(line[5]), int(line[8]), int(line[3])))

#### Verify Query

In [10]:
select_query = """ SELECT artist, song, length FROM songs WHERE sessionId =338 AND itemInSession=4"""
rows = session.execute(select_query)
for row in rows:
    print(row)

Row(artist='Faithless', song='Music Matters (Mark Knight Dub)', length=495.30731201171875)


### Query 2: Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182

We will be filtering based on userId and sessionId columns, so we need to define them as partition columns. And we will sort the results by itemInSession, so we need it as clustering column

#### Create Table

In [11]:
drop_usersongs = """DROP TABLE IF EXISTS usersongs"""
session.execute(drop_usersongs)

create_usersongs = """CREATE TABLE IF NOT EXISTS usersongs (artist text, song text, firstName text, lastName text, itemInSession int, userId int,sessionId int, primary key((userId, sessionId), itemInSession))"""
session.execute(create_usersongs)

#### Insert Records

In [12]:
with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        query = "INSERT INTO usersongs (artist, song, firstName, lastName, itemInSession, userId, sessionId) "
        query = query + "VALUES (%s,%s,%s,%s,%s, %s, %s)"
        session.execute(query, (line[0], line[9], line[1], line[4],int(line[3]), int(line[10]), int(line[8])))

#### Verify Query

In [13]:
select_query = """SELECT artist, song, firstName, lastName from usersongs WHERE userId =10 AND sessionId=182 ORDER BY itemInSession"""
rows = session.execute(select_query)
for row in rows:
    print(row)

Row(artist='Down To The Bone', song="Keep On Keepin' On", firstname='Sylvie', lastname='Cruz')
Row(artist='Three Drives', song='Greece 2000', firstname='Sylvie', lastname='Cruz')
Row(artist='Sebastien Tellier', song='Kilometer', firstname='Sylvie', lastname='Cruz')
Row(artist='Lonnie Gordon', song='Catch You Baby (Steve Pitron & Max Sanna Radio Edit)', firstname='Sylvie', lastname='Cruz')


### Query 3: Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'

#### Create Table

There are no filtering and ordering columns in this query. But in order to identify each row uniquely, we need a unique combination of columns. The combination of (song, firstName, lastName) can uniquely identify each row, but that would mean we are selecting entire table which is not efficient. So we have to introduce userId column, even though it is not used in the query.

In [14]:
## TO-DO: Query 3: Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'
drop_userhistory = """DROP TABLE IF EXISTS userhistory"""
session.execute(drop_userhistory)
create_userhistory = """CREATE TABLE IF NOT EXISTS userhistory (song text, userId int, firstName text, lastName text, primary key(song, userId))"""
session.execute(create_userhistory)

#### Insert Records

In [15]:
with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
#         print(line)
## TO-DO: Assign the INSERT statements into the `query` variable
        query = "INSERT INTO userhistory (song, userId, firstName, lastName) "
        query = query + "VALUES (%s,%s,%s,%s)"
        ## TO-DO: Assign which column element should be assigned for each column in the INSERT statement.
        ## For e.g., to INSERT artist_name and user first_name, you would change the code below to `line[0], line[1]`
        session.execute(query, (line[9], int(line[10]), line[1], line[4]))

#### Verify Query

In [16]:
## TO-DO: Add in the SELECT statement to verify the data was entered into the table
select_query = """ SELECT firstName, lastName FROM userhistory WHERE song='All Hands Against His Own'"""
rows = session.execute(select_query)
for row in rows:
    print(row)

Row(firstname='Jacqueline', lastname='Lynch')
Row(firstname='Tegan', lastname='Levine')
Row(firstname='Sara', lastname='Johnson')


### Drop the tables and close out the sessions

In [17]:
session.execute(drop_songs)
session.execute(drop_usersongs)
session.execute(drop_userhistory)

### Close the session and cluster connection¶

In [18]:
session.shutdown()
cluster.shutdown()